**Example notebook on implementation of tde_spectra_fit**

In [ ]:
# import tde_spectra_fit class:

import numpy as np
from tde_spectra_fit import TDE_fit # this is the module that does the spectral fitting
from BarniolDuranSEM import BarniolDuranSEM # this is the module that applies the Barniol Duran model to estimate physical system parameters

Set up your data arrays of the radio observations. In this example we will use data from Alexander et al 2016.

In [ ]:


# Example data (from Alexander et al 2016):

flux_density = np.array([2.18, 2.12, 2.13, 2.00, 1.84, 1.56, 1.26, 1.06, 0.84,\
               0.73, 0.59, 0.44, 0.30]) #mJy

u_flux_density_up = np.array([0.08, 0.10, 0.09, 0.05, 0.03, 0.03, 0.03, 0.02, 0.04,\
                 0.02, 0.02, 0.09, 0.04])

u_flux_density_low = u_flux_density_up

frequency = np.array([1.4, 1.5, 1.8, 2.6, 3.4, 5.0, 7.1, 8.5, 11.0, 13.5, 16.0,\
            19.2, 24.5]) #GHz

# Also enter the known system properties:

dL = 90 # luminosity distance, in Mpc
z = 0.0206 # redshift
t = 246 # time since jet began in days

# Finally if we know something about the quiescent radio emission of the galaxy, enter it here. If not just enter None.

quiescent_flux_density = 1.8*(frequency/1.4)**(-1) # formula is what Alexander et al 2016 assume

Initialise the spectral fitting class

In [ ]:
S = TDE_fit(fd = flux_density,
        fd_err_low = u_flux_density_low,
        fd_err_up = u_flux_density_up,
        frequency = frequency,
        break_number = 5, # see Granot & Sari 2002, ApJ, 568, 2, Figure 1 for choice of break numbers
        quiescent_flux_density = quiescent_flux_density,
        name = 'Alexander_2016',
        nsteps = 1000, # number of steps for emcee fitting to do
        nwalkers = 400 # number of walkers for emcee fitting to use
           ) 

In [ ]:
# plot initial data to check all is good:

S.plot_initial_data()

In [ ]:
# Looks fine so let's move onto running the emcee fitting and analysing the output:

Fvb, vb, p, Fp, vp, Fvb_u, vb_u, p_u, Fp_u = S.do_fit()

So our fit to the spectrum looks pretty good in the above plot. We find that p=2.8, vp = 3.64, and Fp = 1.15, which are similar to the values reported in Table 2 of Alexander et al 2016 of p=3, vp=4, Fp = 1.14.

Now use the SEM module to calculate the physical jet properties based on this fitted lightcurve:

In [ ]:
# initialise the class
sem = BarniolDuranSEM( 
            vp=4, #GHz
            vp_err=0.06,
            Fvp=1.14, #mJy
            Fvp_err=0.01,
            p=3,
            dL=dL, #luminosity distance
            z=z, #redshift
            t=t, #days
            eps_e=0.1,
            eps_B=None,
            fA=None,
            fV=None,
            LF=None,
            LF_m=None,
            geo='spherical', #options are spherical or conical
            emitting_prop=0.1,# option to assume that the emission emanates from a shell with a thickness of emitting_prop of the blastwave radius.
            va_gtr_vm=True,
            va=None,
            vm=None,
            hot_protons=True,
            save=False,
            name=None,
        )
sem.to_string()

In [ ]:
print(str(sem.eps_e * sem.E))
print(str(sem.eps_e * 6/11 * sem.E))
print(str(sem.fA))
print(str(sem.fV))
print(str(sem.LF_m))